In [1]:
import os

In [2]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier'

## 1. Update config.yaml

```yaml
training:
  root_dir: models/training
  trained_model_path: models/training/model.h5
```

## 3. Update params.yaml

## 3.1 Update src/constant/\_\_init\_\_.py

## 4. Update the entity

In [51]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_epochs: int
    params_learning_rate: float
    params_activation: str
    

## 5. Update the configuration manager src/config/configuration.py

In [52]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.utils.common import read_yaml, create_directories
from src.logger import CustomLogger

class ConfigurationManager:
    def __init__(
        self,logger: CustomLogger,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.logger = logger
        self.config = read_yaml(config_filepath, logger=self.logger)
        self.params = read_yaml(params_filepath, logger=self.logger)    

        create_directories([self.config.model_artifacts_root], logger=self.logger)
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        
        create_directories([training.root_dir], logger=self.logger)
        
        prepare_training_model_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            
            params_augmentation = self.params.AUGMENTATION,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            params_epochs = self.params.EPOCHS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_activation=self.params.ACTIVATION
        )
        return prepare_training_model_config

## 6. Update the components \[data preprocessing, model training, and so on\]

In [68]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from src.logger import CustomLogger
import math

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self, get_base_model_logger: CustomLogger):
        try:
            self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
            self.model.compile(
                optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
                loss="categorical_crossentropy",
                metrics=["accuracy"]
                )

            
            get_base_model_logger.save_logs(msg=f"Model loaded from {self.config.updated_base_model_path} successfully.", log_level="info")
        
        except Exception as e:
            get_base_model_logger.save_logs(msg=f"Error loading model from {self.config.updated_base_model_path}. Error: {str(e)}", log_level="error")
            raise e
        
    def train_valid_generator(self, train_valid_generator_logger: CustomLogger):
    
        try:
            
            
            ########## kwargs start ########## 
            
            datagenerator_kwargs = dict(
                rescale=1./255,
                validation_split=0.2
            )
            
            dataflow_kwargs = dict(
                target_size = self.config.params_image_size[:-1],
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )
            
            val_specific_kwargs = dict(
                directory = self.config.training_data,
                subset="validation",
                shuffle = False,
                class_mode = "categorical"
            )
            
            train_specific_kwargs = dict(
                directory = self.config.training_data,
                subset = "training",
                shuffle = True,
                class_mode = "categorical"
            )
            
            augmentation_kwargs = dict(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)
            
            ########## kwargs end ########## 
            

                    
            ########## validation datagenerator start ########## 
            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
            
            self.valid_generator = valid_datagenerator.flow_from_directory(
                **val_specific_kwargs,
                **dataflow_kwargs
            )
            
            train_valid_generator_logger.save_logs(msg=f"Generated Validation generator successfully with these \ndatagenerator_kwargs: {datagenerator_kwargs},\ndataflow_kwargs: {val_specific_kwargs, dataflow_kwargs}", log_level='info')
            
            ########## validation datagenerator end ##########
            
            ########## training datagenerator start ##########
            if self.config.params_augmentation:
                train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**augmentation_kwargs, **datagenerator_kwargs)
                

            else:
                train_datagenerator = valid_datagenerator
                
            self.train_generator = train_datagenerator.flow_from_directory(
                **train_specific_kwargs,
                **dataflow_kwargs
                )
            
            if self.config.params_augmentation:
                train_valid_generator_logger.save_logs(msg=f"Generated Train generator with AUGMENTATION: True successfully with these \naugmentation_kwargs: {augmentation_kwargs},\ndatagenerator_kwargs: {datagenerator_kwargs}, \ndataflow_kwargs: {train_specific_kwargs, dataflow_kwargs}", log_level='info')
            else:
                train_valid_generator_logger.save_logs(msg=f"Generated Train generator with AUGMENTATION: False successfully with these \ndatagenerator_kwargs: {datagenerator_kwargs}, \ndataflow_kwargs: {train_specific_kwargs, dataflow_kwargs}", log_level='info')
            
            ########## training datagenerator end ##########
        except Exception as e:
            train_valid_generator_logger.save_logs(msg=f"Error in creating train and validation Generators. Error: {e}",log_level="error")
            raise e

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model, logger: CustomLogger):
        try:
            model.save(path)
            logger.save_logs(msg=f"Model saved at {path} successfully", log_level="info")
        except Exception as e:
            logger.save_logs(msg=f"Error in saving model at {path}. Error: {e}", log_level="error")
            raise e
        
        
    def train(self, train_logger: CustomLogger, callback_list: list = None):
        self.steps_per_epoch = math.ceil(self.train_generator.samples // self.train_generator.batch_size)
        self.validation_steps = math.ceil(self.valid_generator.samples // self.valid_generator.batch_size)
        try:
            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator,
                callbacks=callback_list
            )
            train_logger.save_logs(msg="model.fit successfully",log_level="info")
            
        except Exception as e:
            train_logger.save_logs(msg=f"Error in model.fit. Error: {e}",log_level="error")     
            raise e       
        
        self.save_model(path=self.config.trained_model_path, model=self.model, logger=train_logger)

## 7. Update the pipeline

In [71]:
from src.logger import create_log_path, CustomLogger
import logging
from tensorflow.keras import callbacks

# path to save the log files
get_base_model_log_file_path = create_log_path("training/get_base_model")
get_base_model_logger = CustomLogger(logger_name="get_base_model", log_filename=get_base_model_log_file_path)
get_base_model_logger.set_log_level(level=logging.INFO)


train_valid_generator_log_file_path = create_log_path("training/train_valid_generator")
train_valid_generator_logger = CustomLogger(logger_name="train_valid_generator", log_filename=train_valid_generator_log_file_path)
train_valid_generator_logger.set_log_level(level=logging.INFO)



train_log_file_path = create_log_path("training/train")
train_logger = CustomLogger(logger_name="train", log_filename=train_log_file_path)
train_logger.set_log_level(level=logging.INFO)

configuration_log_file_path = create_log_path("configuration")
configuration_logger = CustomLogger(logger_name="configuration", log_filename=configuration_log_file_path)
configuration_logger.set_log_level(level=logging.INFO)


############################## ERROR CORRECTION START ##########################
tf.config.run_functions_eagerly(True)
print("Eager Execution:", tf.executing_eagerly())  # Check if enabled


############## callbacks start ################
# Callbacks
# Early Stopping
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Learning Rate Scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1).numpy()

lr_scheduler = callbacks.LearningRateScheduler(scheduler)

# Model Checkpoint
model_checkpoint = callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True, verbose=1
)

# ReduceLROnPlateau
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
)

# TensorBoard
tensorboard = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# TerminateOnNaN
terminate_on_nan = callbacks.TerminateOnNaN()

# ProgbarLogger
progbar_logger = callbacks.ProgbarLogger()

# CSV Logger
csv_logger = callbacks.CSVLogger('training_log.csv', append=True)



callbacks_list = [
    early_stopping,
    lr_scheduler,
    model_checkpoint,
    reduce_lr,
    tensorboard,
    terminate_on_nan,
    #progbar_logger,
    csv_logger
]

############## callbacks end ################

try:
    config = ConfigurationManager(configuration_logger)
    training_config = config.get_training_config()
    
    training = Training(config=training_config)
    training.get_base_model(get_base_model_logger=get_base_model_logger)
    training.train_valid_generator(train_valid_generator_logger=train_valid_generator_logger)
    training.train(train_logger=train_logger, callback_list=callbacks_list)
    
except Exception as e:
    raise e

INFO:configuration:yaml file: config\config.yaml loaded successfully
INFO:configuration:yaml file: params.yaml loaded successfully
INFO:configuration:created directory at: models
INFO:configuration:created directory at: models/training


Eager Execution: True


INFO:get_base_model:Model loaded from models\prepare_base_model\base_model_updated.h5 successfully.


Found 93 images belonging to 2 classes.


INFO:train_valid_generator:Generated Validation generator successfully with these 
datagenerator_kwargs: {'rescale': 0.00392156862745098, 'validation_split': 0.2},
dataflow_kwargs: ({'directory': WindowsPath('data/raw/kidney-ct-scan-image'), 'subset': 'validation', 'shuffle': False, 'class_mode': 'categorical'}, {'target_size': [224, 224], 'batch_size': 32, 'interpolation': 'bilinear'})


Found 372 images belonging to 2 classes.


INFO:train_valid_generator:Generated Train generator with AUGMENTATION: True successfully with these 
augmentation_kwargs: {'rotation_range': 40, 'width_shift_range': 0.2, 'height_shift_range': 0.2, 'shear_range': 0.2, 'zoom_range': 0.2, 'horizontal_flip': True},
datagenerator_kwargs: {'rescale': 0.00392156862745098, 'validation_split': 0.2}, 
dataflow_kwargs: ({'directory': WindowsPath('data/raw/kidney-ct-scan-image'), 'subset': 'training', 'shuffle': True, 'class_mode': 'categorical'}, {'target_size': [224, 224], 'batch_size': 32, 'interpolation': 'bilinear'})


Epoch 1/2
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.5668 - loss: 2.7622 
Epoch 1: val_loss improved from inf to 0.63150, saving model to best_model.h5


11/11 ━━━━━━━━━━━━━━━━━━━━ 219s 19s/step - accuracy: 0.5718 - loss: 2.6938 - val_accuracy: 0.8750 - val_loss: 0.6315 - learning_rate: 0.0100
Epoch 2/2
 1/11 ━━━━━━━━━━━━━━━━━━━━ 3:15 20s/step - accuracy: 0.8750 - loss: 0.5125
Epoch 2: val_loss did not improve from 0.63150
11/11 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.8750 - loss: 0.5125 - val_accuracy: 0.3281 - val_loss: 0.7024 - learning_rate: 0.0100


INFO:train:model.fit successfully
INFO:train:Model saved at models\training\model.h5 successfully


In [50]:
tf.__version__

'2.19.0'

## 8. Update the main.py

## 9. Update the dvc.yaml

## 10. app.py